**Postech IA para Devs - Fase 3**

Tech Challenge

Grupo 4:


*   Marcos Carielo - rm357969
*   Priscila Nitta - rm357392
*   Renato Mello - rm357879
*   Vitor Soares - rm356986

**Definição do Problema**

Executar o fine-tuning de um foundation model, utilizando o dataset "The AmazonTitles-1.3MM". O modelo treinado deverá:

* Receber perguntas com um contexto obtido por meio do arquivo json
“trn.json” que está contido dentro do dataset.
* A partir do prompt formado pela pergunta do usuário sobre o título do
produto, o modelo deverá gerar uma resposta baseada na pergunta do
usuário trazendo como resultado do aprendizado do fine-tuning os
dados da sua descrição.



**Foundation model escolhido FLAN-T5 Small**

Tamanho do modelo: Cerca de 60 milhões de parâmetros.

Vantagens:
* Leve e rápido para treinar.
* Consome menos memória GPU.
* Bom para tarefas simples.

Desvantagens:
* Menor capacidade de generalização.
* Pode não capturar contextos mais complexos ou relações intricadas nos dados.


**Fluxo de Trabalho**

In [ ]:
import sys
sys.modules.pop('importlib_metadata', None)

<module 'importlib_metadata' from '/usr/local/lib/python3.10/dist-packages/importlib_metadata/__init__.py'>

In [ ]:
# Instalação das bibliotecas necessárias
!pip install --upgrade fsspec==2024.10.0 gcsfs==2024.10.0 datasets transformers torch tf-keras \
    langchain langchain-community chromadb sentence-transformers protobuf==4.21.6 tensorflow \
    tensorflow-metadata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking a

In [ ]:
# Importação das bibliotecas
import json
import torch
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from tqdm import tqdm
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Caminho para o arquivo `trn.json` no dataset "The AmazonTitles-1.3MM"
dataset_path = "/content/drive/MyDrive/Data/LF-Amazon-1.3M/trn.json"

In [ ]:
# 1. Preparação do Dataset
# Carregando e processando o arquivo "trn.json"
def load_and_preprocess_dataset(file_path, max_samples=10000):
    processed_data = []
    seen_titles = set()

    with open(file_path, 'r') as file:
        for line in file:
            item = json.loads(line)
            if item.get('content') and item['title'] not in seen_titles:
                seen_titles.add(item['title'])
                processed_data.append(item)
            if len(processed_data) >= max_samples:
                break

    return processed_data

print("Carregando e pré-processando o dataset...")
data = load_and_preprocess_dataset(dataset_path)

Carregando e pré-processando o dataset...


In [ ]:
# 2. Configuração do ChromaDB
# Usar embeddings para indexação
print("Configurando o ChromaDB...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
chroma_store = Chroma(persist_directory="chroma_db", embedding_function=embeddings)

# Função para dividir textos em chunks
def split_into_chunks(text, chunk_size=100):
    """Divide o texto em chunks menores com base no tamanho especificado."""
    if not text.strip():
        return []
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Adicionar dados ao ChromaDB com chunks
print("Adicionando dados ao ChromaDB com chunks...")
for item in data:
    chunks = split_into_chunks(item["content"], chunk_size=100)
    for chunk in chunks:
        chroma_store.add_texts(
            [chunk],
            metadatas=[{"title": item["title"], "source": "Amazon", "uid": item.get("uid", None)}]
        )

# Configurar o retriever para buscar múltiplos chunks
retriever = chroma_store.as_retriever(search_kwargs={"k": 5})


Configurando o ChromaDB...


<ipython-input-7-2cbc12c0d452>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-7-2cbc12c0d452>:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_store = Chroma(persist_directory="chroma_db", embedding_function=embeddings)


Adicionando dados ao ChromaDB com chunks...


In [ ]:
# 3. Preparação do Modelo FLAN-T5 Small
print("Carregando o modelo FLAN-T5 Small...")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

# Configurar o pipeline HuggingFace para integração com LangChain
print("Configurando pipeline HuggingFace...")
flan_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=128,
    num_beams=5,
    early_stopping=True,
    device=0 if torch.cuda.is_available() else -1
)

llm = HuggingFacePipeline(pipeline=flan_pipeline)

Carregando o modelo FLAN-T5 Small...


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


Configurando pipeline HuggingFace...


<ipython-input-8-fac08f32b575>:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=flan_pipeline)


In [ ]:
# 4. Configuração do RetrievalQA
print("Configurando RetrievalQA...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # Combina os documentos em uma única string
    return_source_documents=True  # Retorna os documentos recuperados
)

Configurando RetrievalQA...


In [ ]:
# 5. Avaliação Antes do Fine-Tuning
print("Testando o modelo antes do fine-tuning...")
def ask_question(question):
    """Função para testar o RetrievalQA."""
    response = qa_chain({"query": question})

    print("Pergunta:", question)
    print("Resposta:", response["result"])

    print("\nFontes:")
    for source in response["source_documents"]:
        metadata = source.metadata
        print(f" - Title: {metadata.get('title', 'unknown')}")
        print(f" - Souce: {metadata.get('source', 'unknown')}")
        print(f" - Uid: {metadata.get('uid', 'unknown')}")
        print(f"   Description: {source.page_content}\n")

question_example = "Tell me about The Fifth Mountain?"
ask_question(question_example)

Testando o modelo antes do fine-tuning...
Pergunta: Tell me about The Fifth Mountain?
Resposta: (ii)

Fontes:
 - Title: The Fifth Mountain
 - Souce: Amazon
 - Uid: 0060930136
   Description: WithThe Fifth Mountain, Paulo Coelho turns his talent for spiritual fiction to the story of the Biblical prophet Elijah. Like a blossoming flower, Coelho opens up the brief account of Elijah's flight from Gilead and his time in Zarephath. He deepens the prophet's character by revealing the thoughts, doubts, and discoveries that Elijah must have experienced as he struggled to find his course in life amidst the confusion of war and political turmoil. When being a prophet of the God of the Israelites is like a warrant for your death, concerns about your chosen path are sure to arise. Perhaps

 - Title: Four Corners: History, Land, and People of the Desert Southwest
 - Souce: Amazon
 - Uid: 0060927593
   Description: With its spectacular ancient rock formations, high plateaus and desert valleys, its Na

In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
# 6. Fine-Tuning do Modelo
print("Preparando o dataset para fine-tuning...")
dataset = Dataset.from_dict({
    "input_text": [f"Tell me about {item['title']}" for item in data],
    "target_text": [item['content'] for item in data]
}).train_test_split(test_size=0.1)

def preprocess_function(examples):
    inputs = tokenizer(examples['input_text'], max_length=256, truncation=True, padding="max_length")
    targets = tokenizer(examples['target_text'], max_length=256, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = dataset['train'].map(preprocess_function, batched=True)
val_dataset = dataset['test'].map(preprocess_function, batched=True)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Data/model",  # Diretório onde os modelos treinados e checkpoints serão salvos.
    evaluation_strategy="epoch",  # Avaliação será realizada ao final de cada época durante o treinamento.
    learning_rate=5e-5,  # Taxa de aprendizado usada pelo otimizador durante o treinamento.
    per_device_train_batch_size=8,  # Tamanho do lote (batch size) durante o treinamento, por dispositivo (GPU/CPU).
    per_device_eval_batch_size=8,   # Tamanho do lote durante a avaliação/validação, por dispositivo (GPU/CPU).
    num_train_epochs=4,  # Número total de épocas (passagens completas pelo dataset) para o treinamento.
    weight_decay=0.01,  # Fator de decaimento de peso usado para regularização L2 e evitar overfitting.
    save_total_limit=2,  # Limita o número de checkpoints salvos; mantém os 2 últimos, apagando os mais antigos.
    logging_dir="/content/drive/MyDrive/Data/logs",  # Diretório onde os logs de treinamento serão salvos.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

print("Iniciando o processo de fine-tuning...")
trainer.train()

Preparando o dataset para fine-tuning...


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-16-bd3913aefe5f>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Iniciando o processo de fine-tuning...


Epoch,Training Loss,Validation Loss
1,2.595300,2.504754
2,2.571100,2.497389
3,2.590600,2.492075
4,2.540400,2.490083


TrainOutput(global_step=4500, training_loss=2.5814827745225695, metrics={'train_runtime': 1646.6676, 'train_samples_per_second': 21.862, 'train_steps_per_second': 2.733, 'total_flos': 3346029084672000.0, 'train_loss': 2.5814827745225695, 'epoch': 4.0})

In [ ]:
# 7. Avaliação Após o Fine-Tuning
print("Testando o modelo após o fine-tuning...")
ask_question(question_example)

Testando o modelo após o fine-tuning...
Pergunta: Tell me about The Fifth Mountain?
Resposta: Paulo Coelho is the author of more than twenty books, includingThe Fifth Mountain.

Fontes:
 - Title: The Fifth Mountain
 - Souce: Amazon
 - Uid: 0060930136
   Description: WithThe Fifth Mountain, Paulo Coelho turns his talent for spiritual fiction to the story of the Biblical prophet Elijah. Like a blossoming flower, Coelho opens up the brief account of Elijah's flight from Gilead and his time in Zarephath. He deepens the prophet's character by revealing the thoughts, doubts, and discoveries that Elijah must have experienced as he struggled to find his course in life amidst the confusion of war and political turmoil. When being a prophet of the God of the Israelites is like a warrant for your death, concerns about your chosen path are sure to arise. Perhaps

 - Title: Four Corners: History, Land, and People of the Desert Southwest
 - Souce: Amazon
 - Uid: 0060927593
   Description: With its s